In [ ]:
import os
import requests
from datetime import datetime
from dotenv import load_dotenv  
from openai import AzureOpenAI
from IPython.display import HTML
from src import aoai_helpers as helpers
from src import aoai_model_configs as model_configs

load_dotenv()  

apim_key = os.environ['APIM_KEY']  
apim_endpoint = os.environ['APIM_ENDPOINT']  
version_of_api = os.environ['AOAI_API_VERSION']
bing_subscription_key = os.environ['BING_SEARCH_SUBSCRIPTION_KEY']
bing_search_url = os.environ['BING_SEARCH_ENDPOINT']

aoai_client = AzureOpenAI(azure_endpoint=apim_endpoint, 
                          api_key=apim_key,  
                          api_version=version_of_api
                        )


In [ ]:
request_results_table = helpers.get_web_search_results()

In [ ]:
# headers = {"Ocp-Apim-Subscription-Key": bing_subscription_key}

# # answer_count = None
# # cc = "en-US" # https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/reference/market-codes#country-codes
# result_count = 50 # integer with max value of 50
# result_freshness = "Week" # "Day", "Week", "Month"
# market = "en-US" # https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/reference/market-codes#country-codes
# # page_offset = None
# # promote_result_type = None

# # The actual search query 
# search_term = "What is the preferred soil ph for growing tomatoes in zone 6a?"
# #

# # response_filer = None
# safe_search = "Off" # "Off", "Moderate", "Strict"
# # set_language = None
# text_decorations = True # A Boolean value that determines whether display strings in the results should contain decoration markers such as hit highlighting characters. If true, the strings may include markers. The default is false. To specify whether to use Unicode characters or HTML tags as the markers, see the textFormat query parameter.
# text_format = "HTML" # The type of markers to use for text decorations (see the textDecorations query parameter). HTML — Use HTML tags to mark content that needs special formatting. For example, use <b> tags to highlight query terms in display strings. Default is Raw.

# params = {
#     "count": result_count,
#     "freshness": result_freshness,
#     "mkt": market,
#     "q": search_term,
#     "SafeSearch": safe_search,
#     "textDecorations": text_decorations,
#     "textFormat": text_format,
#     }

# response = requests.get(bing_search_url, headers=headers, params=params)
# response.raise_for_status()
# search_results = response.json()

# header = "<tr><th>URL</th><th>Snippet</th><th>Date Published</th><th>Result Freshness</th></tr>"


# if 'webPages' in search_results and 'value' in search_results['webPages']:
#     rows = "\n".join([f"""<tr>
#                             <td><a href=\"{v["url"]}\">{v["name"]}</a></td>
#                             <td>{v["snippet"]}</td>
#                             <td>{datetime.strptime(v["datePublished"][:-1], '%Y-%m-%dT%H:%M:%S.%f').strftime('%Y-%m-%d')}</td>
#                             <td>{v.get("datePublishedFreshnessText", "Not available")}</td>
#                         </tr>"""
#                 for v in search_results["webPages"]["value"]])
# else:
#     rows = "No results found."

# # Prepend the header to the rows
# rows = header + rows

# # Display the table with HTML mark-up
# HTML("<table>{0}</table>".format(rows))

In [2]:
# def get_web_search_results(answer_count, country_code, result_count, result_freshness, market, page_offset, promote_result_type, query, response_filter, safe_search, set_language, text_decorations, text_format):
#     """
#     Get web search results from Bing.
#     """
#     try:
#         headers = {"Ocp-Apim-Subscription-Key": bing_subscription_key}
#         params = {
#             "answerCount": answer_count,
#             "cc": country_code,
#             "count": result_count,
#             "freshness": result_freshness,
#             "mkt": market,
#             "offset": page_offset,
#             "promote": promote_result_type,
#             "q": query,
#             "responseFilter": response_filter,
#             "safeSearch": safe_search,
#             "setLang": set_language,
#             "textDecorations": text_decorations,
#             "textFormat": text_format,
#         }
#         response = requests.get(bing_search_url, headers=headers, params=params)
#         response.raise_for_status()
#         search_results = response.json()

#         header = "<tr><th>URL</th><th>Snippet</th><th>Date Published</th><th>Result Freshness</th></tr>"


#         if 'webPages' in search_results and 'value' in search_results['webPages']:
#             rows = "\n".join([f"""<tr>
#                                     <td><a href=\"{v["url"]}\">{v["name"]}</a></td>
#                                     <td>{v["snippet"]}</td>
#                                     <td>{datetime.strptime(v["datePublished"][:-1], '%Y-%m-%dT%H:%M:%S.%f').strftime('%Y-%m-%d')}</td>
#                                     <td>{v.get("datePublishedFreshnessText", "Not available")}</td>
#                                 </tr>"""
#                         for v in search_results["webPages"]["value"]])
#         else:
#             rows = "No results found."

#         # Prepend the header to the rows
#         rows = header + rows

#         # Retun the table with HTML mark-up
#         html_table = HTML("<table>{0}</table>".format(rows))
#         return html_table
#     except Exception as e:
#         print("An error occured during the web search.")
#         return e


In [ ]:
# def get_web_search_results(query, bing_subscription_key, bing_search_url, answer_count=None, country_code=None, result_count=None, result_freshness=None, market=None, page_offset=None, promote_result_type=None, response_filter=None, safe_search=None, set_language=None, text_decorations=None, text_format=None):
#     """
#     Executes a web search using the Bing Web Search API v7 and returns the results in an HTML table format.

#     This function sends a request to the Bing Web Search API with the provided query and other optional search parameters.
#     It then processes the API response and formats the search results as an HTML table with headers for URL, Snippet, 
#     Date Published, and Result Freshness. If no results are found or an error occurs, an appropriate message is returned.

#     Args:
#         query (str): The user's search query string.
#         bing_subscription_key (str): The subscription key for accessing the Bing Web Search API.
#         bing_search_url (str): The URL endpoint for the Bing Web Search API.
#         answer_count (int, optional): The number of answers that you want the search API to return.
#         country_code (str, optional): A 2-character country code of the country where the results come from.
#         result_count (int, optional): The number of search results to return in the response.
#         result_freshness (str, optional): Filter search results based on age. Possible values are "Day", "Week", "Month".
#         market (str, optional): The market where the results come from. Typically, mkt is the country where the user is making the request from.
#         page_offset (int, optional): The number of search results to skip before returning results.
#         promote_result_type (list[str], optional): A list of answer types to promote at the top of the search results.
#         response_filter (list[str], optional): A comma-delimited list of answer types to return in the search results.
#         safe_search (str, optional): A filter used to filter webpages for adult content. Possible values are "Off", "Moderate", "Strict".
#         set_language (str, optional): The language to use for user interface strings.
#         text_decorations (bool, optional): A Boolean value that determines whether display strings contain decoration markers such as hit highlighting characters.
#         text_format (str, optional): The format of the response. Possible values are "Raw", "HTML".

#     Returns:
#         IPython.core.display.HTML: An HTML object that contains a table of the search results. Each row of the table represents a single search result, including the URL, a snippet, the date published, and the result freshness.

#     Raises:
#         Exception: If there is an issue with the API request or processing the response, an exception is printed and returned.

#     Example:
#         >>> html_results = get_web_search_results(
#                 query="Python programming",
#                 bing_subscription_key="YOUR_SUBSCRIPTION_KEY",
#                 bing_search_url="https://api.cognitive.microsoft.com/bing/v7.0/search",
#                 result_count=10,
#                 market="en-US",
#                 safe_search="Strict"
#             )
#         >>> display(html_results)

#     Note:
#         To use this function, you must have a valid Bing Web Search API subscription key.
#         Ensure that you are complying with the Bing Web Search API terms of use.
#     """
#     try:
#         headers = {"Ocp-Apim-Subscription-Key": bing_subscription_key}
#         params = {"q": query}  # Start with the required query parameter

#         # Add optional parameters to the params dictionary if they are not None
#         if answer_count is not None:
#             params["answerCount"] = answer_count
#         if country_code is not None:
#             params["cc"] = country_code
#         if result_count is not None:
#             params["count"] = result_count
#         if result_freshness is not None:
#             params["freshness"] = result_freshness
#         if market is not None:
#             params["mkt"] = market
#         if page_offset is not None:
#             params["offset"] = page_offset
#         if promote_result_type is not None:
#             params["promote"] = promote_result_type
#         if response_filter is not None:
#             params["responseFilter"] = response_filter
#         if safe_search is not None:
#             params["safeSearch"] = safe_search
#         if set_language is not None:
#             params["setLang"] = set_language
#         if text_decorations is not None:
#             params["textDecorations"] = text_decorations
#         if text_format is not None:
#             params["textFormat"] = text_format

#         response = requests.get(bing_search_url, headers=headers, params=params)
#         response.raise_for_status()
#         search_results = response.json()

#         header = "<tr><th>URL</th><th>Snippet</th><th>Date Published</th><th>Result Freshness</th></tr>"


#         if 'webPages' in search_results and 'value' in search_results['webPages']:
#             rows = "\n".join([f"""<tr>
#                                     <td><a href=\"{v["url"]}\">{v["name"]}</a></td>
#                                     <td>{v["snippet"]}</td>
#                                     <td>{datetime.strptime(v["datePublished"][:-1], '%Y-%m-%dT%H:%M:%S.%f').strftime('%Y-%m-%d')}</td>
#                                     <td>{v.get("datePublishedFreshnessText", "Not available")}</td>
#                                 </tr>"""
#                         for v in search_results["webPages"]["value"]])
#         else:
#             rows = "No results found."
        
#         # Prepend the header to the rows
#         rows = header + rows

#         # Retun the table with HTML mark-up
#         html_table = HTML("<table>{0}</table>".format(rows))
#         return html_table

#     except Exception as e:
#         print("An error occurred during the web search.")
#         return e